# Transformers
> Building the original Transformers (Vaswani et al) architecture from scratch.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

## About

In this notebook I'll be constructing the Transformers architecture from scratch using the PyTorch module. 

The Transformer (Vaswani et al) was originally proposed in 2018 by a group of Google researchers. Their paper titled "Attention is all you need" tackled sequence modeling problems by adopting the attention mechanism to draw global dependencies between input and output. The Transformer architecture was the first of its kind to be entirely dependent on the self-attention module while disposing the notion of RNNs and convolution.

![](2023-03-07/TransformerArch.png)
Image credits: Vaswani et al

![](2023-03-07/MHArch.png)
Image credits: Vaswani et al

## Import Libraries

In [6]:
# Imports
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
# import torchtext
import matplotlib.pyplot as plt

print(torch.__version__)

1.13.0+cu117
